In [1]:
# messy imports to be tidied up later
import xlutils, xypath
import databaker
import os
import databaker.constants
from databaker.constants import *
import databaker.databakersolo as ds
from databaker.jupybakeutils import HDim, savepreviewhtml, savepreviewhtmlBAGS, writetechnicalCSV, procrows


In [2]:
# input-output
inputfile = "table-a02.xls"
outputfile = "%s.csv" % os.path.splitext(inputfile)[0]
previewfile = "preview.html"


In [3]:
print("Loading %s which has size %d bytes" % (inputfile, os.path.getsize(inputfile)))
tableset = xypath.loader.table_set(inputfile, extension='xls')
tabs = list(xypath.loader.get_sheets(tableset, "*"))
print("Table names: %s" % ", ".join([tab.name  for tab in tabs]))


Loading table-a02.xls which has size 1177600 bytes
Table names: seasonally adjusted, not seasonally adjusted


In [4]:
# Normal place for your recipe work
conversionsegments = [ ]

for tab in tabs:
    obs = tab.filter("MGSL").assert_one().shift(DOWN).fill(RIGHT).fill(DOWN).is_number().is_not_italic()
    dimensions = [ 
        HDim(tab.col('A').one_of(['Male', 'Female', 'All Persons']), 'gender', CLOSEST, ABOVE), 
        HDim(tab.col('A').is_date(), TIME, DIRECTLY, LEFT), 
        HDim(tab.regex("All aged .*"), 'ages', CLOSEST, UP), 
        HDim(tab.filter("Total economically active").fill(LEFT).fill(RIGHT).is_not_blank(), 'indicator', DIRECTLY, ABOVE),
        HDim(tab.name, 'adjusted_yn')
    ]
    conversionsegments.append((tab, dimensions, obs))



In [10]:
#savepreviewhtml(conversionsegments[0], previewfile)

opening file preview.html
table 'seasonally adjusted' written
javascript calculated


In [5]:
import time
t0 = time.time()
print("converting and writing %d conversion segments into %s" % (len(conversionsegments), outputfile))
convertedrows = [ ]
for conversionsegment in conversionsegments:
    print("conversion segment size %d" % len(conversionsegment[2]))
    rows = procrows(conversionsegment)
    convertedrows.append(rows)
print(time.time() - t0)
# Should detect errors and throw up preview with cells marked
# (though actually the errors manifest further upstream, normally)
    
# here we can do further sorting and postprocessing and allocating to different files 

writetechnicalCSV(outputfile, conversionsegments)



converting and writing 2 conversion segments into table-a02.csv
conversion segment size 23480


/home/goatchurch/sensiblecode/src/databaker/databaker/utils.py:47: UserWarning: Couldn't identify date 'blank'
  warnings.warn("Couldn't identify date {!r}".format(date))


conversion segment size 11288
1.3161506652832031
writing 2 conversion segments into table-a02.csv
conversionwrite segment size 23480
conversionwrite segment size 11288
